# Phase 3.1 — Adaptive Time-Windowing

**Pipeline position:** Phase 2.3 RAG Bridge → **Phase 3.1 (this notebook)** → Phase 3.2 Evidence Accumulation → Phase 3.3 Quantum Fusion

## Purpose
Turn a stream of individual network flows into a **temporally coherent evidence window** that expands under high-similarity threat signals and shrinks during benign periods.

## Design Principles
| Concern | Choice | Rationale |
|---------|--------|-----------|
| Buffer structure | `collections.deque(maxlen=1000)` | O(1) amortized append + automatic overflow eviction |
| Pruning strategy | `while`-loop on timestamp | Fires on every `add_flow`; no batch scans |
| Window adaptation | **Linear additive** (+2s / −1s) | Prevents jitter vs multiplicative (×1.5 / ×0.8) factors |
| Schema | 114-dim `FlowRecord` | Exact match to `vectorize_v51` + ChromaDB medoids |
| Thresholds | High ≥ 0.85 → expand \| Low < 0.40 → shrink | Calibrated to RAG cosine-similarity distribution |

## Adaptive Rules
```
top_sim ≥ 0.85  →  target += 2s   (cap: 60s)   # sustained attack → gather more evidence
top_sim <  0.40  →  target -= 1s   (floor: 5s)  # benign period   → purge stale data
0.40 ≤ top_sim < 0.85  →  no change               # ambiguous zone  → hold steady
```

In [5]:
# ── Cell 1: Imports ────────────────────────────────────────────────────────────
import sys
import time
import numpy as np
import pandas as pd
from collections import deque
from dataclasses import dataclass, field
from typing import List, Optional

print(f'Python      : {sys.version.split()[0]}')
print(f'NumPy       : {np.__version__}')
print(f'Pandas      : {pd.__version__}')
print('collections.deque, dataclasses ✅')
print('Phase 3.1 — Adaptive Time-Windowing imports ready.')

Python      : 3.13.9
NumPy       : 2.1.3
Pandas      : 2.2.3
collections.deque, dataclasses ✅
Phase 3.1 — Adaptive Time-Windowing imports ready.


In [6]:
# ── Cell 2: Constants ──────────────────────────────────────────────────────────
#
# VECTOR_DIMS must equal 114 — the exact output dimension of vectorize_v51()
# and the stored ChromaDB medoids from Phase 2.2.
# Any mismatch here will cause an immediate ValueError in Phase 3.2+.
# ─────────────────────────────────────────────────────────────────────────────

VECTOR_DIMS       : int   = 114      # v5.1 schema dimensionality (MUST NOT change)

# ── Window bounds ─────────────────────────────────────────────────────────────
WIN_MIN_S         : float = 5.0      # minimum window (seconds) — floor
WIN_MAX_S         : float = 60.0     # maximum window (seconds) — cap
WIN_DEFAULT_S     : float = 5.0      # starting window size

# ── Adaptive steps (linear, not multiplicative) ───────────────────────────────
WIN_EXPAND_STEP_S : float = 2.0      # +2s when top_sim ≥ SIM_HIGH_THRESH
WIN_SHRINK_STEP_S : float = 1.0      # -1s when top_sim <  SIM_LOW_THRESH

# ── RAG similarity thresholds ─────────────────────────────────────────────────
SIM_HIGH_THRESH   : float = 0.85     # expand window — likely sustained attack
SIM_LOW_THRESH    : float = 0.40     # shrink window — likely benign / stale

# ── Deque capacity ────────────────────────────────────────────────────────────
DEQUE_MAXLEN      : int   = 1000     # hard cap; deque auto-evicts oldest on overflow

# ── Phase 4.2 — Clean Slate Protocol constants ───────────────────────────────
NORMAL_ARCHETYPES        = frozenset({'NORMAL', 'benign', 'normal'})
BENIGN_STREAK_THRESHOLD  : int   = 3       # consecutive high-sim benign flows to trigger flush
BENIGN_SIM_THRESHOLD     : float = 85.0    # top_similarity ≥ this for streak counting

SEP = '─' * 72
print(SEP)
print(f'  VECTOR_DIMS       = {VECTOR_DIMS}  (must match vectorize_v51 output)')
print(f'  WIN_MIN / DEFAULT / MAX  = {WIN_MIN_S}s / {WIN_DEFAULT_S}s / {WIN_MAX_S}s')
print(f'  Expand step       = +{WIN_EXPAND_STEP_S}s  when top_sim ≥ {SIM_HIGH_THRESH}')
print(f'  Shrink step       = -{WIN_SHRINK_STEP_S}s  when top_sim <  {SIM_LOW_THRESH}')
print(f'  Deque maxlen      = {DEQUE_MAXLEN}')
print(f'  NORMAL_ARCHETYPES = {NORMAL_ARCHETYPES}')
print(f'  Clean Slate       = streak ≥ {BENIGN_STREAK_THRESHOLD} @ sim ≥ {BENIGN_SIM_THRESHOLD}%')
print(SEP)
print('Constants locked. Schema: 114-dim v5.1 ✅')

────────────────────────────────────────────────────────────────────────
  VECTOR_DIMS       = 114  (must match vectorize_v51 output)
  WIN_MIN / DEFAULT / MAX  = 5.0s / 5.0s / 60.0s
  Expand step       = +2.0s  when top_sim ≥ 0.85
  Shrink step       = -1.0s  when top_sim <  0.4
  Deque maxlen      = 1000
  NORMAL_ARCHETYPES = frozenset({'NORMAL', 'normal', 'benign'})
  Clean Slate       = streak ≥ 3 @ sim ≥ 85.0%
────────────────────────────────────────────────────────────────────────
Constants locked. Schema: 114-dim v5.1 ✅


In [7]:
# ── Cell 3: FlowRecord Dataclass ───────────────────────────────────────────────
#
# Represents a single network flow that has been:
#   1. Ingested from the raw ocean stream
#   2. Passed through vectorize_v51()  → (114,) float32 feature vector
#   3. Scored by get_rag_context()     → top_similarity from ChromaDB
#
# The 114-dim constraint is enforced at construction time via __post_init__.
# This prevents any schema-drift bugs from silently propagating to Phase 3.2.
# ─────────────────────────────────────────────────────────────────────────────

@dataclass
class FlowRecord:
    """
    Immutable snapshot of one network flow at post-RAG-scoring time.

    Fields
    ------
    timestamp       : wall-clock seconds (time.time()) when flow was observed
    vector          : (114,) float32 — output of vectorize_v51() for this flow
    top_similarity  : rank-1 RAG cosine similarity in [0, 100] (percentage)
    top_archetype   : e.g. 'EXPLOIT', 'SCAN', 'NORMAL'
    top_attack      : e.g. 'xss', 'ddos', 'benign'
    rag_context     : full context_string from get_rag_context() (Phase 3.2 input)
    packet_label    : optional ground-truth label for simulation/evaluation
    """
    timestamp      : float
    vector         : np.ndarray          # strict: shape must be (VECTOR_DIMS,)
    top_similarity : float               # [0.0, 100.0]
    top_archetype  : str  = 'UNKNOWN'
    top_attack     : str  = 'UNKNOWN'
    rag_context    : str  = ''
    packet_label   : str  = 'UNLABELED'  # ground-truth (simulation only)

    def __post_init__(self):
        # ── Schema enforcement ────────────────────────────────────────────────
        vec = np.asarray(self.vector, dtype=np.float32)
        if vec.shape != (VECTOR_DIMS,):
            raise ValueError(
                f'FlowRecord.vector must have shape ({VECTOR_DIMS},), '
                f'got {vec.shape}. '
                f'Check that vectorize_v51() is producing {VECTOR_DIMS}-dim output.'
            )
        object.__setattr__(self, 'vector', vec)  # store as float32 ndarray

    @property
    def sim_fraction(self) -> float:
        """top_similarity as a fraction [0, 1] for threshold comparisons."""
        return self.top_similarity / 100.0


# ── Smoke test ────────────────────────────────────────────────────────────────
# Verify (a) valid 114-dim vec is accepted, (b) wrong dim raises ValueError.
_ok_vec  = np.zeros(VECTOR_DIMS, dtype=np.float32)
_bad_vec = np.zeros(99,          dtype=np.float32)   # old trial schema — must fail

_fr = FlowRecord(
    timestamp      = time.time(),
    vector         = _ok_vec,
    top_similarity = 86.4,
    top_archetype  = 'EXPLOIT',
    top_attack     = 'xss',
    packet_label   = 'exploit_sim',
)
assert _fr.vector.shape == (VECTOR_DIMS,), 'Shape guard failed'
assert _fr.vector.dtype  == np.float32,    'dtype guard failed'
assert abs(_fr.sim_fraction - 0.864) < 1e-4, 'sim_fraction guard failed'
print(f'✅  FlowRecord(114-dim)   accepted — shape={_fr.vector.shape}, dtype={_fr.vector.dtype}')

try:
    _bad = FlowRecord(timestamp=time.time(), vector=_bad_vec, top_similarity=50.0)
    print('❌  99-dim vector was NOT rejected — schema guard broken!')
except ValueError as e:
    print(f'✅  FlowRecord(99-dim)    rejected  — ValueError raised correctly')
    print(f'    → {e}')

print()
print('FlowRecord dataclass: 114-dim schema guard operational ✅')

✅  FlowRecord(114-dim)   accepted — shape=(114,), dtype=float32
✅  FlowRecord(99-dim)    rejected  — ValueError raised correctly
    → FlowRecord.vector must have shape (114,), got (99,). Check that vectorize_v51() is producing 114-dim output.

FlowRecord dataclass: 114-dim schema guard operational ✅


In [8]:
# ── Cell 4: AdaptiveTimeWindow Class ──────────────────────────────────────────
#
# Maintains a sliding time window of FlowRecord objects.
# The window target grows when the RAG bridge detects a sustained high-similarity
# threat, and shrinks during benign/normal traffic.
#
# Data structure:
#   deque(maxlen=1000)  — O(1) append, O(1) popleft, auto-evicts oldest on overflow
#
# Pruning strategy:
#   A `while` loop at the front of add_flow() checks the oldest flow's timestamp.
#   If it has aged beyond target_seconds, popleft() removes it in O(1).
#   This is far more efficient than the list-comprehension rebuild used in the
#   trial code, which was O(n) on every call.
#
# Window adaptation (linear — no jitter):
#   sim ≥ SIM_HIGH_THRESH (0.85) → target += WIN_EXPAND_STEP_S (+2s)  cap  WIN_MAX_S (60s)
#   sim <  SIM_LOW_THRESH  (0.40) → target -= WIN_SHRINK_STEP_S (-1s)  floor WIN_MIN_S  (5s)
#   else                          → target unchanged
#
# Phase 4.2 — Clean Slate Protocol:
#   After BENIGN_STREAK_THRESHOLD (3) consecutive flows whose
#   top_archetype ∈ NORMAL_ARCHETYPES AND top_similarity ≥ BENIGN_SIM_THRESHOLD (85%),
#   the buffer is flushed and window reset to minimum. This severs the
#   Dempster-Shafer accumulation chain from prior attack episodes,
#   eliminating cross-episode false-positive inflation.
# ─────────────────────────────────────────────────────────────────────────────

class AdaptiveTimeWindow:
    """
    Thread-safe sliding time window for IDS flow evidence accumulation.

    Includes Phase 4.2 Clean Slate protocol: consecutive high-confidence
    benign flows trigger a buffer flush + window reset.

    Parameters
    ----------
    initial_seconds : starting window target (default WIN_DEFAULT_S = 5.0)
    maxlen          : hard deque capacity (default DEQUE_MAXLEN = 1000)
    """

    def __init__(
        self,
        initial_seconds : float = WIN_DEFAULT_S,
        maxlen          : int   = DEQUE_MAXLEN,
    ) -> None:
        self._buffer            : deque = deque(maxlen=maxlen)
        self._target_seconds    : float = float(initial_seconds)
        self._expand_count      : int   = 0
        self._shrink_count      : int   = 0
        self._total_processed   : int   = 0
        self._benign_streak     : int   = 0           # Phase 4.2 Clean Slate
        self._clean_slate_count : int   = 0           # how many flushes occurred
        self._last_action       : str   = 'none'      # initialise before first property access

    # ── Public properties ────────────────────────────────────────────────────

    @property
    def target_seconds(self) -> float:
        """Current window target in seconds."""
        return self._target_seconds

    @property
    def buffer_size(self) -> int:
        """Number of FlowRecords currently in the window."""
        return len(self._buffer)

    @property
    def window_state(self) -> str:
        """
        Human-readable state label:
          'STABLE'    — no adaptation event from the last flow
          'EXPANDING' — target grew (high-similarity threat detected)
          'SHRINKING' — target shrank (benign / stale traffic)
          'CAPPED'    — at maximum window (60s)
          'FLOORED'   — at minimum window (5s)
          'FLUSHED'   — Clean Slate triggered (Phase 4.2)
        """
        if   self._last_action == 'flush':
            return 'FLUSHED'
        elif self._target_seconds >= WIN_MAX_S:
            return 'CAPPED'
        elif self._target_seconds <= WIN_MIN_S:
            return 'FLOORED'
        elif self._last_action == 'expand':
            return 'EXPANDING'
        elif self._last_action == 'shrink':
            return 'SHRINKING'
        else:
            return 'STABLE'

    # ── Core method ──────────────────────────────────────────────────────────

    def add_flow(
        self,
        flow           : FlowRecord,
        current_time   : Optional[float] = None,
    ) -> None:
        """
        Ingest one FlowRecord into the window.

        Steps (in order):
          0. Evaluate benign streak for Clean Slate protocol (Phase 4.2).
          1. Adapt window target based on flow.top_similarity (linear ±step).
          2. Prune expired flows from the *left* (oldest) end of the deque
             using a while-loop — O(1) per pop, O(expired) overall.
          3. Append the new FlowRecord to the *right* of the deque.

        Parameters
        ----------
        flow         : FlowRecord — the incoming scored network flow
        current_time : wall-clock seconds (default: time.time()).  Override in
                       simulations to control the clock without real sleeps.
        """
        if current_time is None:
            current_time = time.time()

        self._total_processed += 1

        # Step 0 — Phase 4.2 Clean Slate: benign streak evaluation ─────────────
        if (flow.top_archetype in NORMAL_ARCHETYPES
                and flow.top_similarity >= BENIGN_SIM_THRESHOLD):
            self._benign_streak += 1
        else:
            self._benign_streak = 0

        if self._benign_streak >= BENIGN_STREAK_THRESHOLD:
            # Flush buffer, reset window, sever Dempster chain
            self._buffer.clear()
            self._target_seconds = WIN_MIN_S
            self._benign_streak  = 0
            self._clean_slate_count += 1
            self._last_action = 'flush'
            # Seed with the triggering flow
            self._buffer.append(flow)
            return

        # Step 1 — Adapt window target (linear growth, no jitter) ──────────────
        sim = flow.sim_fraction  # [0.0, 1.0]

        if sim >= SIM_HIGH_THRESH:
            new_target = min(self._target_seconds + WIN_EXPAND_STEP_S, WIN_MAX_S)
            self._last_action = 'expand' if new_target > self._target_seconds else 'none'
            self._target_seconds = new_target
            if self._last_action == 'expand':
                self._expand_count += 1

        elif sim < SIM_LOW_THRESH:
            new_target = max(self._target_seconds - WIN_SHRINK_STEP_S, WIN_MIN_S)
            self._last_action = 'shrink' if new_target < self._target_seconds else 'none'
            self._target_seconds = new_target
            if self._last_action == 'shrink':
                self._shrink_count += 1

        else:
            self._last_action = 'none'  # ambiguous zone — hold steady

        # Step 2 — Prune expired flows from the left ───────────────────────────
        cutoff = current_time - self._target_seconds
        while self._buffer and self._buffer[0].timestamp < cutoff:
            self._buffer.popleft()

        # Step 3 — Append new flow to the right ───────────────────────────────
        self._buffer.append(flow)

    # ── Fusion interface ─────────────────────────────────────────────────────

    def get_context_for_fusion(self) -> List[FlowRecord]:
        """
        Phase 3.2 / Quantum Fusion input interface.

        Returns
        -------
        A list (oldest → newest) of all FlowRecords currently in the window.
        Safe to iterate; does NOT mutate the internal deque.
        """
        return list(self._buffer)

    # ── Diagnostics ──────────────────────────────────────────────────────────

    def stats(self) -> dict:
        """Return a snapshot of window statistics for logging / monitoring."""
        return {
            'target_seconds'    : round(self._target_seconds, 1),
            'buffer_size'       : self.buffer_size,
            'window_state'      : self.window_state,
            'expand_count'      : self._expand_count,
            'shrink_count'      : self._shrink_count,
            'total_processed'   : self._total_processed,
            'benign_streak'     : self._benign_streak,
            'clean_slate_count' : self._clean_slate_count,
        }

    def __repr__(self) -> str:
        return (
            f'AdaptiveTimeWindow('
            f'target={self._target_seconds:.1f}s, '
            f'buf={self.buffer_size}, '
            f'state={self.window_state}, '
            f'streak={self._benign_streak})'
        )


# ── Smoke test ────────────────────────────────────────────────────────────────
_atw = AdaptiveTimeWindow()
assert _atw.target_seconds == WIN_DEFAULT_S,  'Initial target mismatch'
assert _atw.buffer_size    == 0,              'Buffer should start empty'
assert _atw.window_state   in ('STABLE', 'FLOORED'), 'State should be STABLE or FLOORED on init'

# Clean Slate smoke test: 3 consecutive high-sim benign flows should flush
_cs_vec = np.zeros(VECTOR_DIMS, dtype=np.float32)
for i in range(3):
    _cs_flow = FlowRecord(
        timestamp=float(i), vector=_cs_vec, top_similarity=90.0,
        top_archetype='NORMAL', packet_label='Normal',
    )
    _atw.add_flow(_cs_flow, current_time=float(i))

assert _atw.buffer_size == 1, f'Expected 1 flow after Clean Slate flush, got {_atw.buffer_size}'
assert _atw.window_state == 'FLUSHED', f'Expected FLUSHED state, got {_atw.window_state}'
assert _atw._clean_slate_count == 1, 'Clean Slate should have fired once'

print('AdaptiveTimeWindow class defined (with Phase 4.2 Clean Slate protocol).')
print(f'  Initial repr: {AdaptiveTimeWindow()}')
print(f'  Clean Slate test: 3×NORMAL@90% → buffer flushed to 1 flow ✅')
print()
print('Class smoke test passed ✅')


AdaptiveTimeWindow class defined (with Phase 4.2 Clean Slate protocol).
  Initial repr: AdaptiveTimeWindow(target=5.0s, buf=0, state=FLOORED, streak=0)
  Clean Slate test: 3×NORMAL@90% → buffer flushed to 1 flow ✅

Class smoke test passed ✅


In [9]:
# ── Cell 5: Simulation — 10-Packet Stream ─────────────────────────────────────
#
# Simulates a stream where:
#   Packets 1–5  : NORMAL traffic (top_sim = 30%)  → window SHRINKS / holds at floor (5s)
#   Packets 6–10 : EXPLOIT traffic (top_sim = 90%) → window EXPANDS +2s per packet
#
# Expected Evidence Chain:
#   Packet  1 (Normal)  → target = 5.0s  (FLOORED)
#   Packet  5 (Normal)  → target = 5.0s  (FLOORED)
#   Packet  6 (Exploit) → target = 7.0s  (EXPANDING)
#   Packet  7 (Exploit) → target = 9.0s  (EXPANDING)
#   Packet  8 (Exploit) → target = 11.0s (EXPANDING)
#   Packet  9 (Exploit) → target = 13.0s (EXPANDING)
#   Packet 10 (Exploit) → target = 15.0s (EXPANDING)
#
# Simulation clock: packets arrive 0.5s apart (no real sleep).
# Window size after 10 packets = 15s → all packets remain in buffer
# (oldest is at t=0, cutoff = 4.5 - 15 = -10.5s → nothing pruned).
# ─────────────────────────────────────────────────────────────────────────────

SIM_INTERVAL_S  = 0.5     # simulated inter-packet gap (seconds)
SIM_NORMAL_SIM  = 30.0    # similarity% for normal packets  (< SIM_LOW_THRESH × 100)
SIM_EXPLOIT_SIM = 90.0    # similarity% for exploit packets (≥ SIM_HIGH_THRESH × 100)

# ── Stream definition ─────────────────────────────────────────────────────────
STREAM = [
    # (label,          sim%,           archetype,  attack)
    ('Normal',  SIM_NORMAL_SIM,  'NORMAL',  'benign'),  # pkt  1
    ('Normal',  SIM_NORMAL_SIM,  'NORMAL',  'benign'),  # pkt  2
    ('Normal',  SIM_NORMAL_SIM,  'NORMAL',  'benign'),  # pkt  3
    ('Normal',  SIM_NORMAL_SIM,  'NORMAL',  'benign'),  # pkt  4
    ('Normal',  SIM_NORMAL_SIM,  'NORMAL',  'benign'),  # pkt  5
    ('Exploit', SIM_EXPLOIT_SIM, 'EXPLOIT', 'xss'),     # pkt  6
    ('Exploit', SIM_EXPLOIT_SIM, 'EXPLOIT', 'xss'),     # pkt  7
    ('Exploit', SIM_EXPLOIT_SIM, 'EXPLOIT', 'xss'),     # pkt  8
    ('Exploit', SIM_EXPLOIT_SIM, 'EXPLOIT', 'xss'),     # pkt  9
    ('Exploit', SIM_EXPLOIT_SIM, 'EXPLOIT', 'xss'),     # pkt 10
]

# ── Run simulation ────────────────────────────────────────────────────────────
window     = AdaptiveTimeWindow(initial_seconds=WIN_DEFAULT_S)
rng        = np.random.default_rng(42)
sim_t0     = 0.0   # anchor: packet 1 arrives at t=0

SEP = '─' * 72
print(SEP)
print('  Phase 3.1 — Adaptive Time-Window Simulation   (10-packet stream)')
print(SEP)
print(f'  {"Pkt":<4}  {"Label":<8}  {"Sim%":<6}  '
      f'{"target_s":<10}  {"buf_size":<10}  {"window_state":<12}  {"Δ target"}')
print(SEP)

prev_target = WIN_DEFAULT_S

for pkt_num, (label, sim_pct, archetype, attack) in enumerate(STREAM, start=1):

    # Simulated wall-clock (no real sleep)
    sim_now = sim_t0 + (pkt_num - 1) * SIM_INTERVAL_S

    # Build a 114-dim FlowRecord with a random unit vector (realistic stand-in
    # for vectorize_v51 output; actual inference uses the real vectorizer)
    raw_vec = rng.standard_normal(VECTOR_DIMS).astype(np.float32)
    raw_vec /= np.linalg.norm(raw_vec)          # unit normalise

    flow = FlowRecord(
        timestamp      = sim_now,
        vector         = raw_vec,
        top_similarity = sim_pct,
        top_archetype  = archetype,
        top_attack     = attack,
        rag_context    = f'[RAG CONTEXT] top match: {archetype}/{attack} sim={sim_pct:.1f}%',
        packet_label   = label,
    )

    window.add_flow(flow, current_time=sim_now)

    delta = window.target_seconds - prev_target
    delta_str = (
        f'+{delta:.1f}s  🔺 EXPANDING' if delta > 0
        else f'{delta:.1f}s  🔻 SHRINKING' if delta < 0
        else '  0.0s  ── stable'
    )
    # Annotate floor/cap boundaries
    if window.target_seconds <= WIN_MIN_S and delta <= 0:
        delta_str = '  0.0s  ⬛ FLOORED'

    print(
        f'  {pkt_num:<4}  {label:<8}  {sim_pct:<6.1f}  '
        f'{window.target_seconds:<10.1f}  {window.buffer_size:<10}  '
        f'{window.window_state:<12}  {delta_str}'
    )

    prev_target = window.target_seconds

print(SEP)

# ── Final fusion context snapshot ──────────────────────────────────────────────
fusion_buffer = window.get_context_for_fusion()
print()
print(f'📦  get_context_for_fusion()  →  {len(fusion_buffer)} FlowRecords ready for Phase 3.2')
print()

# Breakdown of buffer contents
n_normal  = sum(1 for f in fusion_buffer if f.packet_label == 'Normal')
n_exploit = sum(1 for f in fusion_buffer if f.packet_label == 'Exploit')
avg_sim   = np.mean([f.top_similarity for f in fusion_buffer])
print(f'  Buffer breakdown  →  Normal: {n_normal}  |  Exploit: {n_exploit}')
print(f'  Average sim% in window  →  {avg_sim:.1f}%')
print(f'  Final window target     →  {window.target_seconds:.1f}s')
print(f'  Final window state      →  {window.window_state}')

# ── Stats snapshot ────────────────────────────────────────────────────────────
print()
print('🔬  Window Stats:')
for k, v in window.stats().items():
    print(f'    {k:<18} : {v}')

────────────────────────────────────────────────────────────────────────
  Phase 3.1 — Adaptive Time-Window Simulation   (10-packet stream)
────────────────────────────────────────────────────────────────────────
  Pkt   Label     Sim%    target_s    buf_size    window_state  Δ target
────────────────────────────────────────────────────────────────────────
  1     Normal    30.0    5.0         1           FLOORED         0.0s  ⬛ FLOORED
  2     Normal    30.0    5.0         2           FLOORED         0.0s  ⬛ FLOORED
  3     Normal    30.0    5.0         3           FLOORED         0.0s  ⬛ FLOORED
  4     Normal    30.0    5.0         4           FLOORED         0.0s  ⬛ FLOORED
  5     Normal    30.0    5.0         5           FLOORED         0.0s  ⬛ FLOORED
  6     Exploit   90.0    7.0         6           EXPANDING     +2.0s  🔺 EXPANDING
  7     Exploit   90.0    9.0         7           EXPANDING     +2.0s  🔺 EXPANDING
  8     Exploit   90.0    11.0        8           EXPANDING     +

In [10]:
# ── Cell 6: Evidence Chain Visualisation ──────────────────────────────────────
#
# Replays the simulation and renders an ASCII bar chart of window target over
# the 10-packet stream — visualising the "waking up" effect as the exploit wave
# arrives at packet 6.
# ─────────────────────────────────────────────────────────────────────────────

# Re-run the simulation to collect per-packet snapshots
window_v2  = AdaptiveTimeWindow(initial_seconds=WIN_DEFAULT_S)
rng_v2     = np.random.default_rng(42)    # same seed → reproducible vectors
snapshots  = []

for pkt_num, (label, sim_pct, archetype, attack) in enumerate(STREAM, start=1):
    sim_now = 0.0 + (pkt_num - 1) * SIM_INTERVAL_S
    raw_vec = rng_v2.standard_normal(VECTOR_DIMS).astype(np.float32)
    raw_vec /= np.linalg.norm(raw_vec)
    flow = FlowRecord(
        timestamp=sim_now, vector=raw_vec, top_similarity=sim_pct,
        top_archetype=archetype, top_attack=attack, packet_label=label,
    )
    window_v2.add_flow(flow, current_time=sim_now)
    snapshots.append({
        'pkt'       : pkt_num,
        'label'     : label,
        'sim_pct'   : sim_pct,
        'target_s'  : window_v2.target_seconds,
        'buf_size'  : window_v2.buffer_size,
        'state'     : window_v2.window_state,
    })

# ASCII bar chart
BAR_SCALE = 2      # chars per second of window target
SEP_VIZ   = '═' * 72

print(SEP_VIZ)
print('  Phase 3.1 — Evidence Chain: Window Target Growth (seconds)')
print(SEP_VIZ)
print(f'  {"Pkt":<4}  {"Type":<8}  {"Window bar (1 char = 0.5s)"}')
print()

DIVIDER_DRAWN = False
for s in snapshots:
    bar_len = int(s['target_s'] * BAR_SCALE)
    colour  = '█' if s['label'] == 'Exploit' else '░'
    bar     = colour * bar_len
    tag     = '' if s['label'] == 'Normal' else f"  ← +2s  {s['state']}"

    # Draw the threshold boundary before packet 6
    if s['pkt'] == 6 and not DIVIDER_DRAWN:
        print(f'  {"─"*4}  {"─"*8}  ← EXPLOIT WAVE ARRIVES ──────────────────────')
        DIVIDER_DRAWN = True

    print(f'  Pkt {s["pkt"]:<2}  {s["label"]:<8}  |{bar:<40}| {s["target_s"]:.1f}s{tag}')

print()
print(SEP_VIZ)
print()

# Phase 3.2 handoff summary
final = snapshots[-1]
print('── Phase 3.2 Handoff Summary ─────────────────────────────────────────')
print(f'  Final window target   : {final["target_s"]:.1f}s')
print(f'  Final window state    : {final["state"]}')
print(f'  FlowRecords in buffer : {final["buf_size"]} (ready for Quantum Fusion)')
print(f'  Schema check          : {VECTOR_DIMS}-dim ✅  (matches vectorize_v51 + ChromaDB)')
print(f'  Adaptation model      : Linear (+{WIN_EXPAND_STEP_S:.0f}s / -{WIN_SHRINK_STEP_S:.0f}s), '
      f'no jitter ✅')
print(f'  Buffer pruning        : while-loop O(1) popleft ✅  (no list-comprehension rebuild)')
print()
print('Phase 3.1 complete. AdaptiveTimeWindow ready for Phase 3.2 integration. ✅')

════════════════════════════════════════════════════════════════════════
  Phase 3.1 — Evidence Chain: Window Target Growth (seconds)
════════════════════════════════════════════════════════════════════════
  Pkt   Type      Window bar (1 char = 0.5s)

  Pkt 1   Normal    |░░░░░░░░░░                              | 5.0s
  Pkt 2   Normal    |░░░░░░░░░░                              | 5.0s
  Pkt 3   Normal    |░░░░░░░░░░                              | 5.0s
  Pkt 4   Normal    |░░░░░░░░░░                              | 5.0s
  Pkt 5   Normal    |░░░░░░░░░░                              | 5.0s
  ────  ────────  ← EXPLOIT WAVE ARRIVES ──────────────────────
  Pkt 6   Exploit   |██████████████                          | 7.0s  ← +2s  EXPANDING
  Pkt 7   Exploit   |██████████████████                      | 9.0s  ← +2s  EXPANDING
  Pkt 8   Exploit   |██████████████████████                  | 11.0s  ← +2s  EXPANDING
  Pkt 9   Exploit   |██████████████████████████              | 13.0s  ← +2s  EXPANDI